In [1]:
###需要改的地方！！！！！： 文件目录地址，数据输入的样本大小，训练的批次轮次
#导入各种程序包库
from __future__ import print_function
import keras
from sklearn.cross_validation import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import utils as np_utils
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Convolution2D, MaxPooling2D
import csv
import numpy as np
import pandas as pd
import os
import sys

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#输入文件的地址，和每个数据样本包含的数据点个数num=1024=32x32,后面的四通道也是32x32
#  num ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
#输入文件的地址，和每个数据样本包含的数据点个数num
def init_data(csv_files,num):
    # df = pd.read_csv(csv_files,engine='python')
    data =[]
    with open(csv_files,encoding='utf-8') as csvfile:
        read_CSV = read_CSV = csv.reader(csvfile)
        for row in read_CSV:
            data.append(row)
        #print(row,i,data)
        data = np.array(data) #将列表转变成数组
        # print(data.shape)
        # print(type(data))
        #判断是否可以被num（样本长度）整除
        if data.shape[1]%num == 0:
            data=data.reshape(-1,num)
        else:
            #除法向下取整，判断一个csv文件之中能取多少个样本
            a=data.shape[1]//num
            # print(a)
            #对数据进行切片处理，取样本数的整数倍数据，后面的不要了
            data=data[:,:a*num]
            #对数据进行分行，形成数组，一行是一个样本
            if data.shape[1]%num == 0:
                data=data.reshape(-1,num)
    return data

In [3]:
#得到 n 个good.csv
def get_goodcsv_files(parent_dir,sub_dirs):
    csv_files = []
    for l, sub_dir in enumerate(sub_dirs):
        csv_path = os.path.join(parent_dir, sub_dir)
        for (dirpath, dirnames, filenames) in os.walk(csv_path):
            for filename in filenames:
                if filename.endswith('good.csv') :
                    filename_path = os.sep.join([dirpath, filename])
                    csv_files.append(filename_path)
    return csv_files

#得到 n 个wrong.csv
def get_wrongcsv_files(parent_dir,sub_dirs):
    csv_files = []
    for l, sub_dir in enumerate(sub_dirs):
        csv_path = os.path.join(parent_dir, sub_dir)
        for (dirpath, dirnames, filenames) in os.walk(csv_path):
            for filename in filenames:
                if filename.endswith('wrong.csv') :
                    filename_path = os.sep.join([dirpath, filename])
                    csv_files.append(filename_path)
    return csv_files

In [9]:
### 读取文件夹下的所有csv文件
######os.listdir(r'D:/TJU/学习/GITHUB/18#右旋-151016-100908.csv')
#  父目录 可以更改！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
parent_dir=r'D:\TJU\学习\GITHUB\test'
#  子目录  可以更改！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
sub_dirs=r'good_csv',r'error_csv'


In [10]:
csv_goodfiles = get_goodcsv_files(parent_dir,sub_dirs)
csv_goodfiles =  np.array(csv_goodfiles)
print(np.array(csv_goodfiles).shape)
print(csv_goodfiles)

csv_wrongfiles = get_wrongcsv_files(parent_dir,sub_dirs)
csv_wrongfiles =np.array(csv_wrongfiles)
print(csv_wrongfiles.shape)
print(csv_wrongfiles)

(112,)
['D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-0-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-2-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-3-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-4-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-5-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-left-good-1-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-right-6-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-right-7-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-right-8-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\15-right-9-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-left-10-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-left-13-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-left-14-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-left-good-11-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-left-good-12-good.csv'
 'D:\\TJU\\学习\\GITHUB\\test\\good_csv\\16-right-good-15-good.csv'
 'D:\\TJU\\学习\\GIT